In [1]:
from pymongo import MongoClient
from pprint import pprint, pp

In [46]:
client = MongoClient('mongodb://localhost:27017/')
db = client.restaurant
restaurants = db.restaurants
print('connexion success')

connexion success


# Exercice  Aggréger des restaurants

# Q : La répartition des restaurants par quartier ?

In [15]:
resto_borough = {"$group":{"_id": "$borough","nb_restaurant":{"$sum" : 1}}}
tri = {"$sort" : {"_id":1}}
for restaurant in restaurants.aggregate([resto_borough,tri]):
    print(restaurant)

{'_id': 'Bronx', 'nb_restaurant': 2338}
{'_id': 'Brooklyn', 'nb_restaurant': 6085}
{'_id': 'Manhattan', 'nb_restaurant': 10258}
{'_id': 'Missing', 'nb_restaurant': 51}
{'_id': 'Queens', 'nb_restaurant': 5656}
{'_id': 'Staten Island', 'nb_restaurant': 969}


# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?

In [17]:
match_grade_C = { "$match" : {"grades.0.grade":"C"}}
proj = {"$project" : {"name":1, "borough":1,"_id":0}}
tri = {"$sort" : {"_id":1}}
for restaurant in restaurants.aggregate([match_grade_C,proj,resto_borough, tri]):
    pp(restaurant)

{'_id': 'Bronx', 'nb_restaurant': 27}
{'_id': 'Brooklyn', 'nb_restaurant': 56}
{'_id': 'Manhattan', 'nb_restaurant': 83}
{'_id': 'Queens', 'nb_restaurant': 47}
{'_id': 'Staten Island', 'nb_restaurant': 7}


# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?

In [47]:
eclater_liste = {"$unwind" : "$grades"}
mean_quartier = {"$group" : {"_id":"$borough","moyenne":{"$avg":"$grades.score"}}}
tri = {"$sort": {"moyenne":-1}}
for restaurant in restaurants.aggregate([eclater_liste, mean_quartier, tri]):
    pp(restaurant)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


# Exercice  Aggréger des transactions

In [79]:
db = client.restaurant
transactions = db.transactions

# Q-1. Calculer le montant total des paiements ?

In [32]:
success = {"$match": {"Payment.Success":True}}
total = {"$group":{"_id":"Payment Total","Total":{"$sum":"$Payment.Total"}}}
for transaction in transactions.aggregate([success,total]):
    pp(transaction)

{'_id': 'Payment Total', 'Total': 1230}


# Q-2. Calculer le montant total par transactions (montant total du panier) ?

In [60]:
liste_eclate = {"$unwind" : "$Panier"}
total = {"$group":{"_id":"$TransactionId","Total":{"$sum":"$Panier.price"}}}

for transaction in transactions.aggregate([liste_eclate,total]):
    pp(transaction)

{'_id': 'tran1', 'Total': 410}
{'_id': 'tran2', 'Total': 410}
{'_id': 'tran4', 'Total': 410}
{'_id': 'tran3', 'Total': 410}


# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

In [40]:
success = {"$match": {"Payment.Success":True}}
total = {"$group":{"_id":"$Payment.Type","Total":{"$sum":"$Payment.Total"}}}
for transaction in transactions.aggregate([success,total]):
    pp(transaction)

{'_id': 'Credit-Card', 'Total': 820}
{'_id': 'Debit-Card', 'Total': 410}


# Q-4. Trouver l'Id le plus élevé.

In [59]:
tri = {"$sort": {"_id": -1}}
limit = {"$limit": 1}
for transaction in transactions.aggregate([tri,limit]):
    pp(transaction)

{'_id': ObjectId('6010003278f09db3ad0385b2'),
 'Id': 102,
 'Name': 'Margaret',
 'TransactionId': 'tran3',
 'Panier': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}],
 'Subscriber': True,
 'Payment': {'Type': 'Credit-Card', 'Total': 410, 'Success': True}}


# Q-5. Find the max price (Transaction.price) ?

In [88]:
tri = {"$sort":{"Panier.price":-1}}
eclate = {"$unwind" : "$Panier"}
limit = {"$limit":1}
filtre = {"$project":{"_id":0,"Panier":1}}
for transaction in transactions.aggregate([eclate,tri,filtre,limit]):
    pp(transaction)

{'Panier': {'ItemId': 'a110', 'price': 210}}


# distinct

In [70]:
db = client.db_agg
people = db.people

# Q : Age unique => distinct ?

In [74]:
for p in people.distinct("age"):
    pp(p)

20.0
35.0
60.0


# Q : Peut-on faire un distinct sur plusieurs key ?

In [77]:
from bson.code import Code
map = Code("function () {"
            "  this.tags.forEach(function(z) {"
            "    emit(z, 1);"
            "  });"
            "}")

reduce = Code("function (key, values) {"
               "  var total = 0;"
               "  for (var i = 0; i < values.length; i++) {"
               "    total += values[i];"
               "  }"
               "  return total;"
               "}")

result = people.map_reduce(map, reduce, "myresults")
for doc in result.find():
    pp(doc)


OperationFailure: MapReduce internal error :: caused by :: TypeError: this.tags is undefined :
@:1:17
, full error: {'ok': 0.0, 'errmsg': 'MapReduce internal error :: caused by :: TypeError: this.tags is undefined :\n@:1:17\n', 'code': 139, 'codeName': 'JSInterpreterFailure'}